<a href="https://colab.research.google.com/github/merucode/RL/blob/92-Project-Trader-Custom_environment/02_Reinforce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 1. Install Dependency

In [5]:
!git clone -b 92-Project-Trader-Custom_environment https://github.com/merucode/RL.git
!cd RL && mv * ../

fatal: destination path 'RL' already exists and is not an empty directory.
mv: cannot stat '*': No such file or directory


In [2]:
!pip install -e trader_world

Obtaining file:///content/trader_world
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 26.0 MB/s eta 0:00:00
  Attempting uninstall: pygame
    Found existing installation: pygame 2.5.0
    Uninstalling pygame-2.5.0:
      Successfully uninstalled pygame-2.5.0
  Running setup.py develop for trader-world


### NOTE: After inistall gym-examples, Restart runtime

# STEP 2. Ready Data

In [3]:
!pip install pyupbit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 3.7 MB/s eta 0:00:00


In [1]:
### Ready Data
import time
from datetime import datetime, timedelta
import pandas as pd

import pyupbit

ticker = 'KRW-BTC'
interval = 'minute5'
start_date = datetime.strptime('2021-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')

df_to = pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume'])

for _ in range(30):
    to = datetime.strftime(start_date, "%Y-%m-%d %H:%M:%S")
    count = 180
    df = pyupbit.get_ohlcv(ticker=ticker,interval=interval,to=to,count=count)
    df = df[df.columns[:-1]]
    start_date += timedelta(hours=15) # hours=3(1m), hours=15(5m)

    df_to = pd.concat([df_to,df])
    time.sleep(0.5)

# print(df_to)
df_to = df_to.reset_index(drop=True)
df = df_to[df_to.columns[:4]]/10000
df = pd.concat([df, df_to[df_to.columns[-1:]]], axis=1)
print(df)

        open    high     low   close     volume
0     3199.2  3199.7  3195.7  3195.9  17.041450
1     3195.9  3197.6  3193.3  3194.3  18.890478
2     3193.3  3194.3  3179.8  3180.8  50.900129
3     3180.8  3188.4  3175.1  3187.3  61.697074
4     3188.0  3197.5  3186.1  3195.2  18.563247
...      ...     ...     ...     ...        ...
5395  4017.9  4022.1  4015.5  4022.0  15.316342
5396  4022.0  4022.4  4008.0  4008.0  36.836909
5397  4007.8  4015.2  4002.2  4015.2  27.802065
5398  4009.7  4018.5  4009.7  4014.1  38.511329
5399  4014.1  4019.5  4011.9  4019.5  27.235054

[5400 rows x 5 columns]


# STEP 3. Setting ENV

In [2]:
import time
import matplotlib.pyplot as plt
import gym

#env = gym.make("trader_world:TraderWorld-v0", df=df, df_render=df, obs_len=180, actions=180, render_mode="rgb_array")
env = gym.make("trader_world:TraderWorld-v0", df=df, df_render=df, obs_len=180, actions=12)

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (180, 5)
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
reset_obs = env.reset()
#env.action_space, env.observation_space
#action = env.action_space.sample()
#action
#obs, reward, done, info = env.step(5)
#print(reward, done, info)
#img = env.render()
#plt.figure()
#plt.imshow(img)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:137: UserWarning: WARN: The obs returned by the `reset()` method was expecting a numpy array, actual type: <class 'list'>
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:226: UserWarning: WARN: Casting input x to numpy array.
  logger.warn("Casting input x to numpy array.")


In [4]:
env.observation_space, env.action_space

(Box(0, 9223372036854775805, (180, 5), int64), Discrete(12))

In [6]:
env.observation_space.shape[0]

180

In [7]:
env.action_space.n

12

# STEP 4. Reinforce